In [ ]:
!pip install diffusers accelerate xformers uuid --quiet

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler, UniPCMultistepScheduler
import random
from uuid import uuid4

In [ ]:
def generate_image_name():
  return str(uuid4()) + ".png"

print(generate_image_name())

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
)
pipe.safety_checker = None
# pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

In [ ]:
pipe.enable_vae_tiling()
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
# 4k 2160x3840
# 2k 1080x2048
# full hd 1080x1920
# hd 720x1280
# 480p 480x640
number_of_images = 2
height = 720
width = 1280
steps = 20

prompt = "a beautiful landscape photograph of a beach in summer"
prompts = [prompt] * number_of_images
images = pipe(prompt=prompts,height=height, width=width,num_inference_steps=steps).images
for image in images:
  image_name = generate_image_name()
  image.save(image_name)
  print("showing: " + image_name)
  display(image)

In [ ]:
# use this to find out what to put in height and width
import math

num_list = []

for i in range(512,2049,8):
  num_list.append(float(i))

# print(num_list)

limit_volume = 1080*1920
ratio = 1080.0/1920.0
max_vol = 0
h = 0
w = 0

for i in num_list:
  vol = i * (i*ratio)
  if(vol > max_vol and vol <= limit_volume):
    max_vol = vol
    h = i
    w = i * ratio

height = h
width = math.floor(w / 8) * 8
print(height)
print(width)
print(width/height)

In [ ]:
!nvidia-smi